In [49]:
from dotenv import load_dotenv
import os

from typing import TypedDict, Annotated
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langgraph.prebuilt import ToolNode, InjectedState, tools_condition
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.types import Command
from langchain_core.tools import tool, InjectedToolCallId


from gen_test_cases import generate_test_cases_from_requirements
from gen_requirements import generate_requirements_from_doc 

load_dotenv()
llm_api_key = os.getenv("GOOGLE_API_KEY")

In [50]:

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens = None,
    api_key = llm_api_key,
)

In [51]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    project_name: str
    context: str
    requirements: list
    testCases: list


In [52]:
@tool
def generate_requirements_from_document_pdf_tool(
    x: Annotated[str, "the PATH to the pdf document from which the tool will generate requirements"],
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """
    Extracts a brief summary and a list of requirements from a project document.

    **Inputs:**
    - `x` (str): The PATH to the pdf document
    
    **Behavior:**
    - Decodes the PDF document from input path into base64 string.
    - Analyzes the document content to extract project requirements.
    - Generates a summary of the document.

    **Outputs:**
    - Updates `state["context"]` with the extracted summary.
    - Updates `state["requirements"]` with a structured list of requirements.
    - Adds a success message in `state["messages"]`.

    **Error Handling:**
    - If the PDF document is invalid or cannot be processed, returns an error message.
    """
    requirements, context = generate_requirements_from_doc(x)
    return Command(update={
        "context": context,
        "requirements": requirements,
        "messages": [
            ToolMessage(
                "Successfully created requirements from the document",
                tool_call_id=tool_call_id
            )
        ]
    })


In [53]:
@tool
def generate_testCases_fromRequirements_tool(
    state: Annotated[AgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """
    Generates a list of test cases from provided requirements.

    **Inputs:**
    - `state["requirements"]` (list[dict]): A list of software or system requirements.
    - `state["context"]` (str): A brief summary to provide additional context.

    **Behavior:**
    - If `requirements` or `context` are missing, returns an error message.
    - Processes requirements to generate structured test cases.
    - Returns a list of test cases in `state["testCases"]`.

    **Outputs:**
    - Updates `state["testCases"]` with generated test cases.
    - Adds a success message to `state["messages"]`.
    """
    requirements = state.get('requirements')
    context = state.get('context')
    if not requirements or not context:
        return Command(update={
            "messages": [
            ToolMessage(
                "Requirements or Context missing",
                tool_call_id=tool_call_id
            )
        ]
    })
    testCases_list = generate_test_cases_from_requirements(requirements, context)
    return Command(update={
        "testCases": testCases_list,
        "messages": [
            ToolMessage(
                "Successfully created test cases from the requirements",
                tool_call_id=tool_call_id
            )
        ]
    })

In [54]:

tools = [generate_testCases_fromRequirements_tool,
         generate_requirements_from_document_pdf_tool]
llm_with_tools = llm.bind_tools(tools)
tools_node = ToolNode(tools)

In [55]:

def call_model(state: AgentState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": response}

In [56]:
# def custom_tool_condition(state: AgentState):
#     messages = state["messages"]
#     last_message = messages[-1]
#     if isinstance(last_message, AIMessage) and last_message.tool_calls:
#         return "tools_node"
#     return END

In [57]:
builder = StateGraph(AgentState)
builder.add_node("call_model", call_model)
builder.add_node("tools", tools_node)

builder.add_edge(START,"call_model")
builder.add_conditional_edges(
    "call_model",
    tools_condition,
)
builder.add_edge("tools","call_model")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [58]:
#Initialize state by loading a project and its info (if any)
with open("..\Prompts\main_system_prompt.txt","r") as f:
    main_system_prompt = f.read()
currState = AgentState(messages=[SystemMessage(content=main_system_prompt)],
                       project_name="QA_automation",
                       testCases=[])

<>:2: SyntaxWarning: invalid escape sequence '\P'
<>:2: SyntaxWarning: invalid escape sequence '\P'
C:\Users\HOME\AppData\Local\Temp\ipykernel_13128\1359178662.py:2: SyntaxWarning: invalid escape sequence '\P'
  with open("..\Prompts\main_system_prompt.txt","r") as f:


In [65]:
# Get user Input

pdf_path = '../ReqView-Example_Software_Requirements_Specification_SRS_Document-3-5.pdf'

userInputContent = 'not yet. I want to have a joke firest'
# Actually run
currState['messages'].append(HumanMessage(content=userInputContent))
config = {"configurable": {"thread_id": "3"}}
graph.invoke(currState,config)
currState = graph.get_state(config).values


In [69]:
currState['context']

"Project Context Summary]\n\nThis project involves the development of a requirements management application based on ReqView v1.0 (released in 2015), designed to help users manage requirements for software and system products. The application follows the ISO/IEC/IEEE 29148:2018 standard for Software Requirements Specifications.\n\nThe application is a browser-based tool that runs offline without server connection, compatible with the latest versions of Chrome and Firefox browsers across Windows, Linux, and Mac operating systems. It features a comprehensive GUI with menus, toolbars, and other interface elements for intuitive keyboard and mouse control.\n\nCore functionalities include capturing and managing requirements specifications, supporting custom attributes (status, priority, etc.), establishing traceability between requirements, and enabling analysis of requirement coverage, gaps, and change impacts. Users can comment on and review requirements, as well as filter and search throu

In [68]:
tempt_requirements = currState['requirements']
tempt_context = currState['context']

In [71]:
tempt_test_case = generate_test_cases_from_requirements(tempt_requirements, tempt_context)

content=[{'type': 'text', 'text': 'You are tasked with generating comprehensive test matrices based on a list of requirements. This is a critical step in ensuring thorough testing coverage for the project. Your goal is to create exhaustive test cases that cover all aspects of the project requirements.\nHere is the requirement list you will be working with and the context behind it:'}, {'type': 'text', 'text': "[{'id': 'REQ-001', 'description': 'The application shall run in the latest versions of Chrome and Firefox browsers on Windows, Linux and Mac operating systems.', 'category': 'Technical Requirements', 'dependencies': 'N/A'}, {'id': 'REQ-002', 'description': 'The application shall allow users to capture and manage requirements specifications.', 'category': 'Functional Requirements', 'dependencies': 'N/A'}, {'id': 'REQ-003', 'description': 'The application shall support custom attributes for capturing additional requirement properties (source, status, priority, verification method, 

UnboundLocalError: cannot access local variable 'output_testCase_list' where it is not associated with a value